In [1]:
# import torch
import pandas as pd
import json,os,sqlite3

In [2]:
def insert_by_batch(con,command,array):
	cur = con.cursor()
	for batch in [array[i:i+2000] for i in range(len(array))[::2000]]:
		cur.executemany(command,batch)
	con.commit()
con = sqlite3.connect("index.db")
cur = con.cursor()
charges = json.load(open("data/documents/common_charge.json",encoding="utf-8"))
query_related = {v for key in charges for v in charges[key][:100] if v.endswith(".json")}
with open("data/query.json",encoding="utf-8") as f:
	for i in f.readlines():
		t =  json.loads(i)
		if t["path"].endswith(".json"):
			query_related.add(t["path"])
data = []
for item in query_related:
	obj = json.load(open("data/documents/documents/{}".format(item),encoding="utf-8"))
	obj["path"] = item
	data.append(obj)

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import tqdm

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

In [14]:
from torch import nn
from torch.utils.data import Dataset
import torch
class Label():
	other = 0
	ajjbqk_b = 1
	ajjbqk_i = 2
	cpfxgc_b = 3
	cpfxgc_i = 4
	pjjg_b = 5
	pjjg_i =6
def get_input(data,tokenizer):
	l = tokenizer(data["qw"],add_special_tokens=True,return_offsets_mapping=True,max_length=20000,
                            padding="max_length",)
	return ( torch.tensor(l["input_ids"],dtype=torch.int32).view(20000,1),get_label(data,l["offset_mapping"][1:]).view(20000,1))
def get_label(data,offset_mapping):
	try:
		a = data["qw"].index(data["ajjbqk"].strip())
		b = data["qw"].index(data["cpfxgc"].strip())
		c = data["qw"].index(data["pjjg"].strip())
	except Exception:
		return None
	ret = [Label.other for i in range(a)]
	ajjbqk = [Label.ajjbqk_b]
	ajjbqk.extend(Label.ajjbqk_i for i in range(len(data["ajjbqk"])))
	ret.extend(ajjbqk)
	m = [Label.other for i in range((b - a-len(data["ajjbqk"])))]
	if m is not None:
		ret.extend(m)
	cpfxgc = [Label.cpfxgc_b]
	cpfxgc.extend([Label.cpfxgc_i for i in range(len(data["cpfxgc"]))])
	ret.extend(cpfxgc)
	m = [Label.other for i in range((c - b-len(data["cpfxgc"])))]
	if m is not None:
		ret.extend(m)

	pjjg = [Label.pjjg_b]
	pjjg.extend([Label.pjjg_i for i in range(len(data["pjjg"]))])
	ret.extend(pjjg)
	ret.extend([Label.other for i in range((len(data["qw"]) - c-len(data["pjjg"])))])
	l2 = []
	l2.append(Label.other)
	for i in offset_mapping:
		if(i[1]==0 and i[0]==0):break
		if i[1] -i[0]>1:
			for j in ret[i[0]:i[1]]:
				if j % 2 ==1: # _b都是奇数
					t = j
					break
			else:
				t = ret[i[0]]
		else:
			t = ret[i[0]]
		l2.append(t)
	if len(l2) < 20000:
		l2.extend([Label.other for i in range(20000 - len(l2))])
	return torch.tensor(l2,dtype=torch.float)
class TrainDataset(Dataset):
	def __init__(self,tokenizer, data):
		self.data = data
		self.tokenizer = tokenizer
	def __len__(self):
		return len(self.data)

	def __getitem__(self, idx):
		return get_input(self.data[idx],self.tokenizer)

device = torch.device("cuda")
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [7]:
ok_data = []
for i in tqdm.tqdm(data[:4000]):
	if len(i["qw"]) > 19000:
		continue
	a,b = get_input(i,tokenizer)
	if b is None:
		continue
	ok_data.append(i)

100%|██████████| 4000/4000 [01:10<00:00, 56.51it/s]


In [94]:
l["offset_mapping"][0]

(0, 0)

In [17]:
from torch.utils.data import DataLoader
batch_size=2
train_dataset = TrainDataset(tokenizer,ok_data)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, drop_last=True)

In [18]:
import time
def train(train_loader, learn_rate=0.001, hidden_dim=256, EPOCHS=5):
    input_dim = 1
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            x = x
            label=label
            counter += 1
            h = h.data
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

In [98]:
for x, label in train_loader:
	break

In [19]:
# model = train(train_loader)

Starting Training of {} model


c:\Users\QiuQichen\.conda\envs\cs224n\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
c:\Users\QiuQichen\.conda\envs\cs224n\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([2, 20000, 1])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (2) must match the size of tensor b (20000) at non-singleton dimension 1

In [62]:
a = data[23]["qw"].index(data[23]["ajjbqk"])
b = data[23]["qw"].index(data[23]["cpfxgc"])
c = data[23]["qw"].index(data[23]["pjjg"])
t.index(1)

334

In [29]:
583+len(a["ajjbqk"])

5251

In [26]:
a["qw"].index(a["cpfxgc"])

5251

In [27]:
a["qw"].index(a["pjjg"])

5880

In [20]:
data

,ajId,ajName,ajjbqk,cpfxgc,pjjg,qw,writId,writName,path
0,c988b11b-3543-474e-8137-254ad23fcfcb,戴琪、杨绍宝协助组织卖淫一案,余干县人民检察院指控，2017年12月18日温馨99休闲会所开业以来，该会所老板（另案处理...,本院认为，被告人戴琪、杨绍宝明知他人利用休闲会所实施组织卖淫犯罪活动，仍受其雇佣在卖淫场所实...,一、被告人戴琪犯协助组织卖淫罪，判处有期徒刑一年六个月，并处罚金人民币五千元； （刑期自判...,江西省余干县人民法院 刑事判决书 （2018）赣1127刑初91号 公诉机关江西省余干县人民...,0654f9f6a68e2e5c1eceae9a75192910bdb,戴琪、杨绍宝协助组织卖淫一审刑事判决书,c988b11b-3543-474e-8137-254ad23fcfcb/0654f9f6a...
1,29043223-7716-40f9-b685-f6376617dac9,前郭尔罗斯蒙古族自治县人民检察院指控原审被告人初志刚犯贪污罪、滥用职权罪、逃税罪，原审被告单...,原公诉机关前郭县人民检察院前检刑诉（2009）158号起诉书指控： （一）2004-200...,本院认为，关于原审被告人初志刚在经营松原市兴达商厦期间，出售货架子等物品得款9700元，是否...,一、撤销前郭尔罗斯蒙古族自治县人民法院（2010）前刑重初字第3号刑事判决。 二、原审被告...,吉林省松原市宁江区人民法院 刑事判决书 （2014）宁刑再初字第1号 原公诉机关前郭尔罗斯蒙...,0460b7a3bd2db32f1769b8e94a768845651,前郭尔罗斯蒙古族自治县人民检察院指控原审被告人初志刚犯贪污罪、滥用职权罪、逃税罪，原审被告单...,29043223-7716-40f9-b685-f6376617dac9/0460b7a3b...
2,dda855e8-04ba-43b9-bd44-b2ee8970c23e,陆云健掩饰、隐瞒犯罪所得、犯罪所得收益罪一案,盐城市亭湖区人民检察院指控，2016年9月2日至4日期间，被告人陆云健明知是犯罪所得的赃款...,本院认为，被告人陆云健明知是犯罪所得的赃款帮助转移，情节严重，其行为已构成掩饰、隐瞒犯罪所得...,一、被告人陆云健犯掩饰、隐瞒犯罪所得罪，判处有期徒刑四年八个月，并处罚金人民币三万元。 刑...,江苏省盐城市亭湖区人民法院 刑事判决书 （2017）苏0902刑初433号 公诉机关盐城市亭...,0205c9764077f5064c745c922c63d42a7f2,陆云健掩饰、隐瞒犯罪所得、犯罪所得收益罪一审刑事判决书,dda855e8-04ba-43b9-bd44-b2ee8970c23e/0205c9764...
3,1449be59-b065-467c-a8c4-4becc5421a6e,李美荣非法制造、销售非法制造的注册商标标识罪一案,公诉机关指控，2015年11月初，一名姓肖的男子（在逃，身份信息不详）向被告人李美荣下单订...,本院认为，被告人李美荣无视国家法律，伪造他人注册商标标识并予以销售，情节特别严重，其行为已构...,一、被告人李美荣犯非法制造注册商标标识罪，判处有期徒刑三年，并处罚金人民币三千元，限于本判...,李美荣非法制造、销售非法制造的注册商标标识罪一审刑事判决书 广东省深圳市宝安区人民法院 刑事...,03136ce9ab43779243e41a19f27b25e3346,李美荣非法制造、销售非法制造的注册商标标识罪一审刑事判决书,1449be59-b065-467c-a8c4-4becc5421a6e/03136ce9a...
4,b3dcc279-fc01-464d-b1e7-3b65836342e2,陈少平、钟翔虚假一案,兴国县人民检察院指控：2013年，被告人陈少平、钟翔、应强、刘斌合伙在兴国县潋江镇潋江大道...,本院认为：被告人陈少平、钟翔、应强、刘斌共同以捏造的事实提起民事诉讼，妨害司法秩序，并严重侵...,一、被告人陈少平犯虚假诉讼罪，判处有期徒刑一年八个月，并处罚金人民币三万元； （刑期从判决...,江西省兴国县人民法院 刑事判决书 （2019）赣0732刑初118号 公诉机关江西省兴国县人...,065bf6fc03a06ed67da63c9d06c8cc8d13a,陈少平、钟翔虚假诉讼一审刑事判决书,b3dcc279-fc01-464d-b1e7-3b65836342e2/065bf6fc0...
...,...,...,...,...,...,...,...,...,...
8876,b3df73c2-4e71-4701-bcbb-641856f6b0b5,马某某贪污、玩忽职守一案,定西市安定区人民检察院指控：2008年6月，武威某某职业培训学校（以下简称武威某某学校，系...,本院认为，被告人马某某身为国家机关工作人员，在担任武威职业学院院长期间，作为学院管理国家助学...,一、被告人马某某犯玩忽职守罪，判处有期徒刑一年，犯受贿罪，判处有期徒刑一年，决定执行有期徒...,甘肃省定西市安定区人民法院 刑事判决书 （2013）安刑初字第187号 公诉机关定西市安定区...,06049b712e804ace3c52d06c556b8d231d7,马某某贪污、玩忽职守一审刑事判决书,b3df73c2-4e71-4701-bcbb-641856f6b0b5/06049b712...
8877,48caedf6-be2b-4ca3-85f9-112f885d6a16,郑军、邓勇滥伐林木一案,经审理查明，2017年5月至2018年12月，被告人郑军、邓勇在未办理林木采伐许可证的情况...,本院认为：被告人郑军、邓勇违反森林法的规定，在未取得林木采伐许可证的情况下，任意采伐林木，数...,一、被告人郑军犯滥伐林木罪，判处有期徒刑一年二个月，缓刑二年，并处罚金四千元。 （缓刑考验...,湖北省秭归县人民法院 刑事判决书 （2019）鄂0527刑初59号 公诉机关秭归县人民检察院...,0525bab6dc1a03c0c662032e44c7fd3beb0,郑军、邓勇滥伐林木一审刑事判决书,48caedf6-be2b-4ca3-85f9-112f885d6a16/0525bab6d...
8878,5d8c7cf8-9ac8-4974-a2bc-2e8e6c15120d,被告人李文宇犯故意毁坏财物罪一案,经审理查明：2016年5月11日23时许，被告人李文宇在其位于绵阳市涪城区长虹世纪城二期3...,本院认为：被告人李文宇故意毁坏他人财物，数额较大，其行为已触犯国家刑律，构成故意毁坏财物罪。...,被告人李文宇犯故意毁坏财物罪，判处罚金人民币5000元（罚金限在判决生效之日起三个月内缴纳...,四川省绵阳市涪城区人民法院 刑事判决书 （2016）川0703刑初319号 公诉机关绵阳市涪...,00509a674e888856d93b64a947ba1d16a2f,被告人李文宇犯故意毁坏财物罪一案一审刑事判决书,5d8c7cf8-9ac8-4974-a2bc-2e8e6c15120d/00509a674...
8879,b4480369-4194-484a-9996-81e58e8b2977,裴春红非法持有毒品罪一案,NaN,辩护意见 被告人裴某某对起诉书指控的犯罪事实无异议，并认罪。 查明事实 经依法审查查明，2...,被告人裴某某犯非法持有毒品罪，判处有期徒刑十个月，并处罚金人民币六千元（刑期自2016年1...,裴春红非法持有毒品罪一审刑事判决书 {C} 广东省深圳市宝安区人民法院 刑事判决书 （201...,0540a9871eb8fd0a4af9bdbd2bf37bad148,裴春红非法持有毒品罪一审刑事判决书,b4480369-4194-484a-9996-81e58e8b2977/0540a9871...
